In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [49]:
# Read the CSV file into a DataFrame
teams_stats = pd.read_csv('../basketballPlayoffs/teams_stats_encoded.csv')
season_11 = pd.read_csv('../basketballPlayoffs/season_11.csv')

In [50]:
print(season_11.head())

  tmID  year  points_prev1  oRebounds_prev1  dRebounds_prev1  rebounds_prev1  \
0  ATL    11     94.731339        13.209410        28.510020       41.719431   
1  CHI    11     82.624878        11.635651        29.559256       41.194907   
2  CON    11     87.613941         9.865952        21.769437       31.635389   
3  IND    11    100.483479        10.813904        28.287799       39.101702   
4  LAS    11     79.624237        10.446219        29.985909       40.432128   

   assists_prev1  steals_prev1  blocks_prev1  turnovers_prev1  ...  \
0      20.354342     10.699971      3.938426        17.914609  ...   
1      14.867777      7.110676      4.760039        17.747307  ...   
2      18.016086      7.613941      2.895442        16.675603  ...   
3      17.817194     11.809469      5.767415        17.542555  ...   
4      22.433067      8.341945      4.847346        17.961484  ...   

   fgMade_prev1  ftAttempted_prev1  ftMade_prev1  threeAttempted_prev1  \
0     36.805112         

In [56]:
from pycaret.classification import *

# Assuming you've set up your environment and imported your data into a variable named teams_stats

# Set up the classification problem
classification = setup(data=teams_stats, target='playoff', ignore_features=['tmID', 'year', 'confID'], session_id=69)

# Create a LightGBM model
lgbm_model = create_model('gbc')

tuned_lgbm=tune_model(lgbm_model, n_iter=50, choose_better=True)

print(f"Evaluating model: {tuned_lgbm}")

# You can access feature importances using get_config function
feature_importance = get_config('X_train').columns.tolist()

# Calculate the 'rating' column based on multipliers
season_11['rating'] = sum(season_11[column] * lgbm_model.feature_importances_[i] for i, column in enumerate(feature_importance))

# Filter the results DataFrame by year and conference
year_conf_results = season_11[(season_11['year'] == 11) & (season_11['confID'] == 'EA')]

# Sort the East conference results by 'rating' in descending order and select the top 4 teams
east_teams = year_conf_results.sort_values(by='rating', ascending=False).head(4)

# Filter the results DataFrame by year and conference
year_conf_results = season_11[(season_11['year'] == 11) & (season_11['confID'] == 'WE')]

# Sort the West conference results by 'rating' in descending order and select the top 4 teams
west_teams = year_conf_results.sort_values(by='rating', ascending=False).head(4)

print("East Conference Teams:")
print(east_teams)

print("West Conference Teams:")
print(west_teams)

,Description,Value
0,Session id,69
1,Target,playoff
2,Target type,Binary
3,Original data shape,"(126, 22)"
4,Transformed data shape,"(126, 19)"
5,Transformed train set shape,"(88, 19)"
6,Transformed test set shape,"(38, 19)"
7,Ignore features,3
8,Numeric features,18
9,Preprocess,True


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5556,0.7500,0.8000,0.5714,0.6667,0.0526,0.0598
1,0.6667,0.6000,0.8000,0.6667,0.7273,0.3077,0.3162
2,0.6667,0.7000,0.8000,0.6667,0.7273,0.3077,0.3162
3,0.7778,0.8500,1.0000,0.7143,0.8333,0.5263,0.5976
4,0.8889,0.8000,0.8000,1.0000,0.8889,0.7805,0.8000
5,0.5556,0.5500,0.8000,0.5714,0.6667,0.0526,0.0598
6,0.7778,0.7500,0.8000,0.8000,0.8000,0.5500,0.5500
7,0.6667,0.6000,0.6000,0.7500,0.6667,0.3415,0.3500
8,0.8750,0.6667,1.0000,0.8333,0.9091,0.7143,0.7454


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7778,0.8000,1.0000,0.7143,0.8333,0.5263,0.5976
1,0.6667,0.6000,0.8000,0.6667,0.7273,0.3077,0.3162
2,0.6667,0.6000,0.8000,0.6667,0.7273,0.3077,0.3162
3,0.8889,0.8000,1.0000,0.8333,0.9091,0.7692,0.7906
4,0.8889,0.8000,0.8000,1.0000,0.8889,0.7805,0.8000
5,0.4444,0.3500,0.6000,0.5000,0.5455,-0.1538,-0.1581
6,0.6667,0.7500,0.6000,0.7500,0.6667,0.3415,0.3500
7,0.5556,0.7500,0.6000,0.6000,0.6000,0.1000,0.1000
8,0.8750,0.6667,1.0000,0.8333,0.9091,0.7143,0.7454


Fitting 10 folds for each of 50 candidates, totalling 500 fits
Evaluating model: GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.01, loss='log_loss', max_depth=11,
                           max_features='sqrt', max_leaf_nodes=None,
                           min_impurity_decrease=0.002, min_samples_leaf=1,
                           min_samples_split=7, min_weight_fraction_leaf=0.0,
                           n_estimators=190, n_iter_no_change=None,
                           random_state=69, subsample=0.9, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
East Conference Teams:
   tmID  year  points_prev1  oRebounds_prev1  dRebounds_prev1  rebounds_prev1  \
3   IND    11    100.483479        10.813904        28.287799       39.101702   
6   NYL    11    103.432085        12.073784        26.383454       38.457239   
0   ATL    11     94.73133

In [21]:
from pycaret.classification import *

classification = setup(data = teams_stats, train_size=0.8, target = 'playoff', ignore_features=['tmID', 'year', 'confID'], session_id=123, fold=10, feature_selection=True, remove_multicollinearity=True, pca_method=True, normalize=True, transformation=True, fix_imbalance=True, use_gpu=True, experiment_name='basketballPlayoffs')

#best_model = compare_models()
#tuned_model = tune_model(best_model)
#evaluate_model(tuned_model)
#final_model = finalize_model(tuned_model)
#predictions = predict_model(final_model, data=teams_stats)

#svm = create_model('svm')
#tuned_svm = tune_model(svm)
#evaluate_model(tuned_svm)
#final_svm = finalize_model(tuned_svm)
#predictions = predict_model(final_svm, data=teams_stats)

#dt = create_model('dt')
#tuned_dt = tune_model(dt)
#evaluate_model(tuned_dt)
#final_dt = finalize_model(tuned_dt)
#predictions = predict_model(final_dt, data=teams_stats)

rf = create_model('rf')
tuned_rf = tune_model(rf, n_iter=50, choose_better=True)
evaluate_model(tuned_rf)
final_rf = finalize_model(tuned_rf)
predictions = predict_model(final_rf, data=teams_stats)

lr = create_model('lr')
tuned_lr= tune_model(lr, n_iter=50, choose_better=True)
evaluate_model(tuned_lr)
final_lr = finalize_model(tuned_lr)
predictions = predict_model(final_lr, data=teams_stats)

lightgbm = create_model('lightgbm')
tuned_lightgbm= tune_model(lightgbm, n_iter=50, choose_better=True)
evaluate_model(tuned_lightgbm)
final_lightgbm = finalize_model(tuned_lightgbm)
predictions = predict_model(final_lightgbm, data=teams_stats)

gbc = create_model('gbc')
tuned_gbc= tune_model(gbc, n_iter=50, choose_better=True)
evaluate_model(tuned_gbc)
final_gbc = finalize_model(tuned_gbc)
predictions = predict_model(final_gbc, data=teams_stats)

#knn = create_model('knn')
#tuned_knn= tune_model(knn)
#evaluate_model(tuned_knn)
#final_knn = finalize_model(tuned_knn)
#predictions = predict_model(final_knn, data=teams_stats)

#best_models = compare_models(n_select=14)
#tuned_models = [tune_model(i) for i in best_models]
#blended = blend_models(estimator_list = tuned_models, fold=10)
#evaluate_model(blended)
#final_blended = finalize_model(blended)
#predictions = predict_model(final_blended, data=season_11)

'''
# Create all available classification models
best_models = [create_model(model) for model in models().index]

# Tune models
tuned_models = [tune_model(model, n_iter=10, choose_better=True) for model in best_models]

# Evaluate models using predict_model on test data
for model in tuned_models:
    print(f"Evaluating model: {model}")
    predict_model(model, data=season_11)




#results = pull()
'''

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: gfx1035, Vendor: Advanced Micro Devices, Inc.
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the 

,Description,Value
0,Session id,123
1,Target,playoff
2,Target type,Binary
3,Original data shape,"(126, 22)"
4,Transformed data shape,"(140, 4)"
5,Transformed train set shape,"(114, 4)"
6,Transformed test set shape,"(26, 4)"
7,Ignore features,3
8,Numeric features,18
9,Preprocess,True


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: gfx1035, Vendor: Advanced Micro Devices, Inc.
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the 

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.4000,0.5200,0.4000,0.4000,0.4000,-0.2000,-0.2000
1,0.4000,0.3600,0.8000,0.4444,0.5714,-0.2000,-0.3333
2,0.5000,0.7200,0.4000,0.5000,0.4444,0.0000,0.0000
3,0.4000,0.4167,0.3333,0.5000,0.4000,-0.1538,-0.1667
4,0.7000,0.9167,0.6667,0.8000,0.7273,0.4000,0.4082
5,0.4000,0.3333,0.5000,0.5000,0.5000,-0.2500,-0.2500
6,0.5000,0.4583,0.5000,0.6000,0.5455,0.0000,0.0000
7,0.2000,0.0417,0.1667,0.2500,0.2000,-0.5385,-0.5833
8,0.5000,0.6042,0.6667,0.5714,0.6154,-0.0870,-0.0891


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,21:53:52
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Classifier


Fitting 10 folds for each of 50 candidates, totalling 500 fits


KeyboardInterrupt: 